In [8]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import os
from tqdm import tqdm
import tensorflow as tf
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D

class similarimage():
    
    def __init__(self, filepath):
        self.filepath = filepath
        self.model = self.model1()
    
    def model1(self):
        base_model = InceptionV3(weights='imagenet', include_top=False)
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        self.model = Model(inputs=base_model.input, outputs=x)
        return self.model

    def features(self, filepath):
        featurestest = []
        img = cv2.imread(filepath)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (299,299)) 
        img = img/255.0
        img = img.reshape(1,299, 299, 3)
        out = self.model.predict(img)
        featurestest.append(out)
        features1 = np.squeeze(featurestest)
        features=features1.reshape(1,-1)
        return features

def similarity(filepath):
    sim = similarimage(filepath)
    df = pd.read_csv('./features1.csv')
    df1 =df.drop(labels='0', axis=1)
    embeddings = df1.to_numpy()
    dist = (embeddings - sim.features(filepath))**2
    dist = np.sqrt(np.sum(dist, axis=1))
    dfdist = pd.DataFrame({'distance':dist})
    dfdist.sort_values('distance', ascending=True, inplace=True)
    ind=list(dfdist.index)[0]
    return df.iloc[ind][0]

In [11]:
similarity('./images/00a3edd22dc7859c487a64777fc8d093.jpg')

'/home/nisar/Documents/nisar/reverse image/images00a3edd22dc7859c487a64777fc8d093.jpg'